In [1]:
import torch
DEVICE = "cuda:0"

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [3]:
model = model.to(DEVICE)

In [4]:
text = "What is the greatest chicken in the world?"

In [5]:
inputs = tokenizer(text, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model(**inputs,  output_hidden_states=True)
    hidden_states = outputs.hidden_states

In [28]:
import pandas as pd
df = pd.read_csv("angry.csv")
df

,text,angry
0,diagnosis that i have a stomache ulcer,0
1,ive been having more frequent hot flashes thro...,0
2,i was laughing at my husband because he was st...,0
3,i walk into a restaurant well any public place...,0
4,ive just been feeling extremely outcasted and ...,0
...,...,...
545,i feel like the world is just being bitter and...,1
546,i see people who have accomplished so much mor...,1
547,i started to see a concerning pattern i d rush...,1
548,i feel so tortured by it,1


In [39]:
import numpy as np
from tqdm.notebook import tqdm

In [41]:
@torch.no_grad()
def forward(model, tokenizer, text):
  inputs = tokenizer(text, return_tensors="pt").to(model.device)
  outputs = model(**inputs,  output_hidden_states=True)
  return outputs

def extract_activations(layer_output):
  """TODO: consider pooling instead"""
  return layer_output[:, -1, :]

@torch.no_grad()
def df_activations(df):
  new_col = []
  for idx, row in tqdm(df.iterrows(), total=len(df)):
    outputs = forward(model, tokenizer, row["text"])
    activations = get_activations_per_input(model, outputs)
    new_col.append(activations)
  df["activations"] = new_col
  return df

@torch.no_grad()
def get_activations_per_input(model, outputs):
  acts = torch.zeros(model.config.num_hidden_layers, model.config.hidden_size, device=DEVICE)
  for idx, layer_output in enumerate(outputs.hidden_states):
      if idx == 0:
        continue
      acts[idx-1, :] = extract_activations(layer_output)
  acts_ndarray = acts.flatten().cpu().numpy()
  return acts_ndarray

df = df_activations(df)

  0%|          | 0/550 [00:00<?, ?it/s]

In [43]:
df.to_parquet("angry_activations.parquet", index=False)